# .NET Initialization
<div style="width:100%;height:6px;background-color:Black;"></div>

In [47]:
import clr
clr.AddReference(r"..\bin\MiniSim.Core")

import MiniSim.Core.Expressions as expr
from  MiniSim.Core.Flowsheeting import MaterialStream, Flowsheet
import MiniSim.Core.Numerics as num
from MiniSim.Core.UnitsOfMeasure import Unit, SI, METRIC, PhysicalDimension
from MiniSim.Core.ModelLibrary import Flash, Heater, Mixer, Splitter, EquilibriumStageSection
import MiniSim.Core.PropertyDatabase as chemsep
from MiniSim.Core.Reporting import Generator, StringBuilderLogger
from MiniSim.Core.Thermodynamics import ThermodynamicSystem

from ipywidgets import interact, interactive, fixed, interact_manual,FloatSlider
import ipywidgets as widgets

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [48]:
%matplotlib inline

In [49]:
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')
plt.rcParams['savefig.dpi'] = 75

plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14

#plt.rcParams['text.usetex'] = True
#plt.rcParams['font.family'] = "serif"
#plt.rcParams['font.serif'] = "cm"
plt.rcParams['grid.color'] = 'k'

# Set up Thermodynamics
<div style="width:100%;height:6px;background-color:Black;"></div>

In [50]:
Database = chemsep.ChemSepAdapter()
logger = StringBuilderLogger()
reporter = Generator(logger)

sys= ThermodynamicSystem("Test2","NRTL", "default")
sys.AddComponent(Database.FindComponent("Isobutene"))
sys.AddComponent(Database.FindComponent("Methanol"))
sys.AddComponent(Database.FindComponent("N-butane"))
sys.AddComponent(Database.FindComponent("Acetone"))
sys.AddComponent(Database.FindComponent("Chloroform"))

isobutene= sys.Components[0]
methanol= sys.Components[1]
nbutane= sys.Components[2]
acetone= sys.Components[3]
chloroform= sys.Components[4]

Database.FillBIPs(sys)

In [51]:
names=sys.GetComponentIds()
longnames=sys.GetComponentNames()
cas= sys.GetComponentCASNumbers()
mw= sys.GetComponentMolarWeights()
numComps=len(sys.Components)

df_data=pd.DataFrame(zip(longnames, cas, mw), index= names, columns= ["Name","CASNo","MolarWeight"])
df_data

,Name,CASNo,MolarWeight
Isobutene,Isobutene,115-11-7,56.10632
Methanol,Methanol,67-56-1,32.04186
N-butane,N-butane,106-97-8,58.12220
Acetone,Acetone,67-64-1,58.07914
Chloroform,Chloroform,67-66-3,119.37760


# Interactive Plots
<div style="width:100%;height:6px;background-color:Black;"></div>

In [119]:
solver= num.DecompositionSolver(logger)

mixtureBubble= MaterialStream("MixB", sys)
mixtureBubble.Specify("VF",0.0)
mixtureBubble.Specify("P",1000, METRIC.mbar)
for c in names:
    mixtureBubble.Specify("n["+c+"]",1.0)
mixtureBubble.InitializeFromMolarFlows()
mixtureBubble.FlashPZ()

mixtureDew= MaterialStream("MixB", sys)
mixtureDew.Specify("VF",1.0)
mixtureDew.Specify("P",1000, METRIC.mbar)
for c in names:
    mixtureDew.Specify("n["+c+"]",1.0)
mixtureDew.InitializeFromMolarFlows()
mixtureDew.FlashPZ()

flowTx= Flowsheet("plots")
flowTx.AddMaterialStreams(mixtureBubble, mixtureDew);    

In [112]:
def updatePlots(psys, tsys, i,j):   
    numSteps=21     
    fig,ax=plt.subplots(1,1,figsize=(10,10))       
    xvec=[]
    yvec=[]
    yvec2=[]     
    mixtureBubble.Specify("P",psys, METRIC.mbar)
    mixtureDew.Specify("P",psys, METRIC.mbar)
    for c in range(numSteps):
        for k in range(numComps):                    
            mixtureBubble.Specify("n["+names[k]+"]",0.0)
            mixtureDew.Specify("n["+names[k]+"]",0.0)
        mixtureBubble.Specify("n["+i.ID+"]",c/(numSteps-1))
        mixtureBubble.Specify("n["+j.ID+"]",1.0-c/(numSteps-1))
        mixtureBubble.InitializeFromMolarFlows()
        mixtureBubble.FlashPZ()
        mixtureDew.Specify("n["+i.ID+"]",c/(numSteps-1))
        mixtureDew.Specify("n["+j.ID+"]",1.0-c/(numSteps-1))
        mixtureDew.InitializeFromMolarFlows()
        mixtureDew.FlashPZ()
        solver.Solve(flowTx)                               
        xvec.append(mixtureBubble.GetVariable('x['+i.ID+']').Val())               
        yvec.append(mixtureBubble.GetVariable('T').Val()-273.15)               
        yvec2.append(mixtureDew.GetVariable('T').Val()-273.15)               
    ax.plot(xvec, yvec)
    ax.plot(xvec, yvec2)                    
    ax.set_xlabel('$x_{'+i.ID+'}$')       
    ax.set_ylabel('T [°C]')            
    #ax.set_ylim([25,175])
    ax.set_xlim([0,1])
    ax.legend(["Tbub","Tdew"])  
    plt.tight_layout()
    plt.title(f'(T,x)-Diagram of {i.ID}/ {j.ID} at {psys:.1f} mbar');
    return plt


In [120]:
tsys=100
psys=1000
cA=acetone
cB=methanol

In [121]:
def updatePressure(p=1000):
    psys=p
    updatePlots(psys, tsys, cA,cB)      

In [122]:
interact(updatePressure,p=FloatSlider(min=500, max=1e4, value=1000,step=100, continuous_update=False)  );

interactive(children=(FloatSlider(value=1000.0, continuous_update=False, description='p', max=10000.0, min=500…